In [1]:
import os
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm

In [5]:
# Define directories
base_dir = "E://data//single_TARE_geometrymatchedwithresampling(PP_fixed)"
output_csv_path = "E://single_TARE//resampledPP_match_results2.csv" 

In [3]:
def load_image(path):
    return sitk.ReadImage(path)

def image_properties(img):
    return {
        'size': img.GetSize(),
        'origin': img.GetOrigin(),
        'spacing': img.GetSpacing(),
        'direction': img.GetDirection()
    }

In [6]:
results = []

# Loop through patients
for patient_id in tqdm(os.listdir(base_dir)):
    patient_path = os.path.join(base_dir, patient_id)
    if not os.path.isdir(patient_path):
        continue

    try:
        pre_img = load_image(os.path.join(patient_path, 'pre.nrrd'))
        ap_img = load_image(os.path.join(patient_path, 'AP.nrrd'))
        pp_img = load_image(os.path.join(patient_path, 'PP.nrrd'))

        pre_props = image_properties(pre_img)
        ap_props = image_properties(ap_img)
        pp_props = image_properties(pp_img)

        mismatch_info = []

        for key in ['size', 'origin', 'spacing', 'direction']:
            if not (pre_props[key] == ap_props[key] == pp_props[key]):
                mismatch_info.append(
                    f"{key}: pre={pre_props[key]}, ap={ap_props[key]}, pp={pp_props[key]}"
                )

        match_flag = 0 if not mismatch_info else 1
        results.append({
            'PatientID': patient_id,
            'Match': match_flag,
            'MismatchDetail': ' | '.join(mismatch_info)
        })

    except Exception as e:
        print(f"[Error] {patient_id}: {e}")
        results.append({
            'PatientID': patient_id,
            'Match': 1,
            'MismatchDetail': f"Error: {e}"
        })

# Save to CSV
df = pd.DataFrame(results)
df.to_csv(output_csv_path, index=False)
print(f"✅ Matching check completed and saved to {output_csv_path}")


100%|██████████| 144/144 [03:38<00:00,  1.52s/it]

✅ Matching check completed and saved to E://single_TARE//resampledPP_match_results2.csv
